## <font color = "brown"> Importing Libraries </font>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings as wn

wn.filterwarnings("ignore")

In [4]:
def fetch_data():
    
    # Function to fetch market data
    df = pd.read_excel("../../raw_data/market_data.xlsx") # Open original data
    df = df[["Time (UTC+10)", "Regions VIC Trading Price ($/MWh)"]] # Select time and victoria prices
    df.columns = ["Time", "Price"] # Rename columns
    df["Time"] = pd.to_datetime(df["Time"]) # Convert data type
    df = df.sort_values("Time").reset_index(drop = True) # Finalise
    
    return df

## <font color = "brown"> Moving Average Model </font>

The moving average (MA) model calculates the centered moving average line of the time series. This model takes one parameter $n$, which is the number of data points back and forth to be considered in the centered moving average.

The centered moving average in mathematical terms is,

$$
\begin{aligned}
\text{CMA}_n(y_t) &= \frac{y_t + \sum\limits_{i=1}^{n}\left[y_{t-i} + y_{t+1}\right]}{2n + 1} \\
&= \frac{y_t + (y_{t-1} + y_{t-2} + \dots + y_{t-n}) + (y_{t+1} + y_{t+2} + \dots + y_{t+n})}{2n + 1}
\end{aligned}
$$

Where $y_t$ represents the spot price at any time $t$. Hence technically, this average is a $(2n + 1)$ simple moving average for $y_{t+n}$, or the $(2n+1)$ centered moving average for $y_t$. Since we are using it for time $t$, we will call it a centered moving average.


In [5]:
'''
CMA takes:
- df: the original market data frame
- n: the parameter of the number of data points back and forth to be considered in the CMA
and returns the df containing the moving averages of each period
'''

def CMA(df, n):
    
    # Defining variables
    n = n + 1
    series = df["Price"]
    
    # Calculate (n+1)-simple moving average looking back
    BackSum = series.rolling(n).sum()
    series = series.iloc[::-1]
    
    # Calculate (n+1)-simple moving average looking ahead
    FrontSum = series.rolling(n).sum()
    series = series.iloc[::-1]
    
    # Calculate the (2n+1)-centered moving average
    df["MA"] = (BackSum + FrontSum - series) / (2 * n - 1)
    
    return df.dropna().reset_index(drop = True)

In [6]:
'''
VisualiseMA takes the df as input and plots the moving averages
'''

def VisualiseMA(df, ma):
    
    # Visualise the calculated moving averages
    sns.lineplot(y = "Price", x = "Time", data = df)
    sns.lineplot(y = "MA", x = "Time", data = ma)

In [7]:
'''
Divide takes:
- df: market data containing moving averages
processes these by saying if the price is below the moving average, we charge, otherwise, we dischareg
and returns a series of LowerPrice as series that contains actions to discharge and UpperPrice as series that contains actions to chargge
'''

def Divide(df):
    
    # Defining data points above the moving average
    UpperPrice = df[df.Price >= df.MA]
    UpperPrice["Status"] = "Discharge"
    
    # Defining data points below the moving average
    LowerPrice = df[df.Price < df.MA]
    LowerPrice["Status"] = "Charge"
    
    return LowerPrice, UpperPrice

In [8]:
'''
BiggerPicture takes:
- original df
- Charge and Dischareg as outputs from Divide(df)
and assign actions to time points
'''

def BiggerPicture(df, Charge, Discharge):
    
    # Function to assign action to each time point
    cdc = pd.concat([Charge, Discharge], ignore_index = True)

    # Join with original dataframe
    dff = pd.merge(df, cdc[["Time", "Status"]], on = "Time", how = "left")
    
    # Select and rename columns
    dff = dff[["Time", "Price", "Status"]]
    dff.columns = ["Time", "Price", "Status"]
    
    return dff

In [9]:
'''
MovingAverage takes:
- df as the original market time series data
- n as the parameter which is the number of data points back and forth that are included in the CMA formula
does:
1. adds a new column containing calculated CMA
2. adds a new column containing actions to charge or discharge
3. merge this back to original timeline
returns: the df containing actions to charge or dischargeg
'''

def MovingAverage(n, df):
    
    # Create Moving Average Price Data
    PriceMA = CMA(df.copy(), n)
    
    #Visualise Moving Averages (Do not visualise for entire dataset)
    #VisualiseMA(df, PriceMA)
    
    # Divide Price Data
    LowerPrice, UpperPrice = Divide(PriceMA)
    
    # Assign Action to Time Point
    final = BiggerPicture(df.copy(), LowerPrice, UpperPrice)
    
    return final

## <font color = "brown"> Region Maximisation Model </font>

The region maximisation (RM) model takes a region of consecutive actions and the opening capacity, and returns the most effective way to charge or discharge. We define a 'region of consecutive actions' as time periods where the previous models recommend consecutive actions.

<table>
<tr>
    <th> Chronological dispatches </th><th> RM Dispatches</th>
</tr>
<tr>
    <td>
        <table>
            <tr><th> Time </th><th> Action </th><th>  ROCA </th><th>  Price </th><th> Dispatch</tr>
            <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td> \$20 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 00:30:00 </td><td> Charge </td><td> 1 </td><td> \$24 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td> \$23 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 01:30:00 </td><td> Charge </td><td> 1 </td><td> \$21 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:00:00 </td><td> Charge </td><td> 1 </td><td> \$18 </td><td> -44 </td></tr>
            <tr><td>2120-01-01 02:30:00 </td><td> Charge </td><td> 1 </td><td> \$17 </td><td> 0 </td></tr>
            <tr><td>2120-01-01 03:00:00 </td><td> Discharge </td><td> 2 </td><td> \$31 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 03:30:00 </td><td> Discharge </td><td> 2 </td><td> \$30 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 04:00:00 </td><td> Discharge </td><td> 2 </td><td> \$29 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 04:30:00 </td><td> Discharge </td><td> 2 </td><td> \$32 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:00:00 </td><td> Discharge </td><td> 2 </td><td> \$33 </td><td> 117 </td></tr>
            <tr><td>2120-01-01 05:30:00 </td><td> Discharge </td><td> 2 </td><td> \$34 </td><td> 0 </td></tr>
        </table>
    </td>
    <td>
        <table>
            <tr><th> Time </th><th> Action </th><th>  ROCA </th><th>  Price </th><th> Dispatch</tr>
            <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td> \$20 </td><td> -150 </td></tr>
            <tr><td> 2120-01-01 00:30:00 </td><td> Charge </td><td> 1 </td><td> \$24 </td><td> 0 </td></tr>
            <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td> \$23 </td><td> -44 </td></tr>
            <tr><td> 2120-01-01 01:30:00 </td><td> Charge </td><td> 1 </td><td> \$21 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:00:00 </td><td> Charge </td><td> 1 </td><td> \$18 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 02:30:00 </td><td> Charge </td><td> 1 </td><td> \$17 </td><td> -150 </td></tr>
            <tr><td>2120-01-01 03:00:00 </td><td> Discharge </td><td> 2 </td><td> \$31 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 03:30:00 </td><td> Discharge </td><td> 2 </td><td> \$30 </td><td> 117 </td></tr>
            <tr><td>2120-01-01 04:00:00 </td><td> Discharge </td><td> 2 </td><td> \$29 </td><td> 0 </td></tr>
            <tr><td>2120-01-01 04:30:00 </td><td> Discharge </td><td> 2 </td><td> \$32 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:00:00 </td><td> Discharge </td><td> 2 </td><td> \$33 </td><td> 135 </td></tr>
            <tr><td>2120-01-01 05:30:00 </td><td> Discharge </td><td> 2 </td><td> \$34 </td><td> 135 </td></tr>
        </table>
    </td>
</tr>


The left table shows market dispatches formulated using chronological order, while the right table shows dispatch formulated by the RM model. We can see in any of the fictitious table above, how consecutive time points with the same actions are categorised as regions of consecutive actions (ROCA). Changing actions would change the ROCA category, hence the name 'region of **consecutive** actions'. 

In the process of filling up market dispatches, we notice that using chronological order to fill dispatches, most of the time, yields inefficient results. As we can see for both ROCAs in the left table, the 2 best price points in each ROCAs are given the least amount of dispatches. This would decrease the total revenue in the final calculation. The RM model solves this by maximising each ROCAs based on their current capacities, action (charge/discharge) and most importantly, the prices. In other words, it allocates the best price points to the most dispatches. The table on the right shows how the dispatches move, when the RM model is applied.

In [10]:
'''
FindActions takes:
- df from the MovingAverage function
and separates each actions as a ROCA (Region of Consecutive Actions)
'''

def FindActions(df):
    
    # Separate periods of action and no action
    action = df[df["Status"] != "Do Nothing"]

    # Group action periods based on consecutive action
    action["Period"] = action["Status"].ne(action["Status"].shift()).cumsum()
    action["Vec"] = np.where(action["Status"] == "Discharge", action["Price"], -action["Price"])
    
    return action

In [11]:
# This function sets the limit of possible dispatches
def Limit(capacity, status, end = 0):
    
    # Limit possible dispatch by looking at capacity and action
    if status == "Charge":
        if capacity + 135 > 580:
            # Maximum charge
            restrict = 580 - capacity
        else:
            restrict = 135
    else:
        if capacity - 150 < end:
            # Maximum discharge
            restrict = end - capacity
        else:
            restrict = -150
            
    return restrict

'''
Chronological takes:
- action as the dataframe containing actions (Charge / Discharge) with its corresponding ROCA
- start as the opening capacity
- end as the enndinn capacity
processes these and fills out the capacity based on chronological order
and returns the df containing dispatches based on chronological order
'''

def Chronological(action, start = 0, end = 0):
    # Allocate dispatch based on chronological order
    
    # Set initial energy conditions
    action["Opening Capacity"] = 0
    action["Closing Capacity"] = 0

    # Declare additional energy variables
    action = action.sort_values("Time").reset_index(drop = True)
    action["Opening Capacity"].loc[0] = start
    action["Restrict"] = np.nan
    
    # Looping to set opening and closing capacities
    for i in action.index:
        data = action.loc[i]
        
        # Limiting possible energy dispatch
        action["Restrict"].loc[i] = Limit(data["Opening Capacity"], data["Status"], end)
        
        # Closing capacity = Opening capacity + activities
        action["Closing Capacity"].loc[i] = action["Opening Capacity"].loc[i] + action["Restrict"].loc[i]
        try:
            # Set next entry's opening capacity to current entry's closing capacity
            action["Opening Capacity"].loc[i + 1] = action["Closing Capacity"].loc[i]
        except:
            # Exception if last data
            pass
        
    action["Actual"] = np.where(action["Status"] == "Discharge", action["Restrict"] * 0.9, action["Restrict"] / 0.9)
    
    return action

In [12]:
def Comprehend(data):
    
    # Assign preferable SPs higher market dispatch
    if data["Restrict"].sum() != 0:
        restrict = sorted(abs(data["Restrict"]).tolist(), reverse = True)
        data = data.sort_values("Vec", ascending = False)
        data["Restrict"] = restrict
    
    return data

'''
Maximise takes:
- action as the dataframe containing actions (Charge / Discharge) with its correspondingn ROCA
and returns the df containing regions that are maximized
by sorting the price in ascending/descending order based on the actions
'''

def Maximise(action):
    
    # Divide dataframe based on consecutive actions
    periods = [v for k, v in action.groupby(["Period"])]
    
    # Looping through each consecutive action
    for i in range(len(periods)):
        
        data = periods[i]
        # Only process if actions are not limited to 0 due to capacity
        #periods = [Comprehend(df) for df in periods]
        
        if data["Restrict"].sum() != 0:
            
            # Assign preferable SPs higher market dispatch
            restrict = sorted(abs(data["Restrict"]).tolist(), reverse = True)
            data = data.sort_values("Vec", ascending = False)
            data["Restrict"] = restrict
            periods[i] = data
            
    # Rejoin all periodic dataframe
    try:
        action = pd.concat(periods)
    except:
        action = pd.DataFrame(columns = action.columns)
    action = action.sort_values("Time").reset_index(drop = True)
    
    # Reassign market dispatch number
    action["Actual"] = -np.where(action["Status"] == "Discharge", action["Restrict"] * 0.9, action["Restrict"] / 0.9)
    
    return action

In [13]:
def PostProcess(action, df):
    
    # Select columns required for revenue calculation
    action = action[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]]
    
    # Reverse the price because we used the opposite sign to maximise
    action["Restrict"] = np.where(action["Status"] == "Charge", abs(action["Restrict"]), -abs(action["Restrict"]))
    action["Actual"] = np.where(action["Status"] == "Charge", -abs(action["Actual"]), abs(action["Actual"]))
    
    # Remove no-dispatch actions
    action = action[action["Restrict"] != 0]
    df = pd.merge(df, action, on = ["Time", "Price"], how = "left")
    
    # Imputation
    df["Status"] = df["Status"].replace(np.nan, "Do Nothing")
    df["Actual"] = df["Actual"].replace(np.nan, 0)
    df["Restrict"] = df["Restrict"].replace(np.nan, 0)
    
    return df

In [14]:
'''
Maximisation takes:
- MX as the data after being processed inn Moving Average model
- df as the original market data set after preprocess of MA
and returns the df containing the Openig Capacity, Dispatch, Closing Capacity and maximized action
(i.e. lowest price of the ROCA that are charged more than higher prices)
'''

def Maximisation(MX, df, start = 0, end = 0, Chronos = True):
    
    # Separate action and no action
    action = FindActions(MX.dropna())
    
    # Fill out dispatch values based on the energy restrictions
    if Chronos:
        action = Chronological(action, start, end)
    
    # Maximise allocated dispatch value
    action = Maximise(action)
    
    # Post processing
    action = PostProcess(action, df)
    
    return action.reset_index(drop = True)

## <font color = "brown"> Loss Removal Model </font>

The loss removal (LR) model looks at transactions between 2 ROCAs and see if there are any matching transactions, where it is actually not profitable to conduct those transactions. This model makes use of the fact that ROCAs are always switching in action by definition. Hence we can compare consecutive ROCAs and find unprofitable transactions.

Due to the MA and RM model, it is almost guaranteed that discharging ROCA's price points are always higher than their preceeding charging ROCA's price points. But due to the efficiency and the Marginal loss factor (MLF) in the revenue calculations, higher alone is not enough.

When charging takes place in a time point $t$, at $C$ Mwh, the actual increase in capacity is only $D = C \times 0.9$ accounting for efficiency. When dispatching this in the next time point, the maximum possible dispatched amount is $D \times 0.9$ accounting to efficiency. At the end, the revenue from charge $C$ is only obtained from $C \times 0.9^2$. We want to know whether the charging-discharging pairs of points from consecutive ROCAs passes the breakeven point or not. If not, we will discard these points.

Let $P_c$ be the charging price, $P_d$ be the discharging price, and $R$ be the revenue. The breakeven point is a price combination point, where the revenue will be 0.

$$
\begin{aligned}
R &= D P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
0 &=  0.9^2 C P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
0.80271 P_d &= \frac{P_c}{0.991} \\
P_d &= 1.257P_c
\end{aligned}
$$

Hence to translate this, we want to filter out every charging-discharging pair where the discharging price $P_d$ is less than $1.257P_c$ and keep if otherwise.

The process of filtering out actions is a process of elimination. Every charging ROCA will have a succeeding discharging ROCA. We will compare these succeding pairs of charges and discharges. We define restricted dispatch $RD$ as the market dispatch, $MD$, multiplied by the efficiency factor.

$$
\begin{aligned}
RD &=
\begin{cases}
       MD\div0.9 & \text{for Action = Charge}  \\
       MD\times0.9 & \text{for Action = Discharge}  \\
\end{cases}
\end{aligned}
$$

The algorithm of this model takes the least profitable pairs of charge $C_t$ and discharge $D_t$, and discard them if not profitable. The process accounts for their $RD$, which means if the value of $RD$ does not match between the compared pairs, we will remove  the action with least $RD$ and subtract the other action by $\min(C_t, D_t)$. The action that remains (first anchor point) will then be compared to the next worst competing action. The process stops if at any comparison, the discharge price exceeds the breakeven point.

In [15]:
'''
FillCapacity takes:
- df with restricted dispatch column and output dfs with calculated capacities.
'''

def FillCapacity(df):

    # Initialise Opening Capacity for dataframe order
    df["Opening Capacity"] = 0
    
    # Filling closing and opening capacities with dispatch given
    df["Closing Capacity"] = round(df["Restrict"].cumsum())
    df["Opening Capacity"] = df["Closing Capacity"].shift()
    df["Opening Capacity"].loc[0] = 0
    
    # Edit actual dispatch based on edited actions
    df["Actual"] = -np.where(df["Status"] == "Discharge", df["Restrict"] * 0.9, df["Restrict"] / 0.9)
    
    return df

'''
LRProcess takes:
- df as the copy of sdf (the processed df after Moving Average and Region Maximisation model)
and preprocesses this before removing unprofitable charge/dischargeg pairings
by adding the region of consecutive actions as ['Period']
'''

def LRProcess(df):
    
    # Preprocessing
    df = df.drop(["Opening Capacity", "Closing Capacity"], axis = 1)
    df = df[df["Status"] != "Do Nothing"]
    df["Period"] = df["Status"].ne(df["Status"].shift()).cumsum()
    
    return df.reset_index(drop = True)

In [16]:
# This function returns the discharge price required to breakeven
def Breakeven(price):
    return price/(0.9 * 0.9 * 0.991 * 0.991)

'''
RemoveLoss takes:
- sdf as the processed df after Moving Average and Region Maximisation model
this is the main function that removes each region of consecutive action's unprofitable charge/dischargeg pairings
'''

def RemoveLoss(sdf):
    
    # Sort charging ROCA based on worst (highest) price
    charge = sdf[sdf["Status"] == "Charge"].sort_values("Price", ascending = False).reset_index(drop = True)
    
    # Sort discharging ROCA based on worst (lowest) price
    discharge = sdf[sdf["Status"] == "Discharge"].sort_values("Price", ascending = True).reset_index(drop = True)

    try:
        # Initialise worst charge and discharge price
        CP = charge["Price"].loc[0]
        DP = discharge["Price"].loc[0]
        
        # Initialise charge and discharge dispatch
        disrestrict = discharge["Restrict"].loc[0]
        charestrict = charge["Restrict"].loc[0]
        
    except KeyError:
        # Exception if last charging ROCA does not have
        # a Succeeding competing ROCA
        return charge
    
    # The order of the first absolute sum determines 
    # which ROCA gets treated as the initial anchor point
    if abs(disrestrict) > abs(charestrict):
        
        # Treats first discharge as the first anchor point
        first = discharge
        second = charge
        pos = "Dis" # For convenience further in
        
    else:
        # Treats first charge as the first anchor point
        first = charge
        second = discharge
        pos = "Char" # For convenience further in
       
    # Iteration Variables
    i = 0
    position = "First"
        
    # Loop stays until breakeven is achieved at any point.
    # By definition, if the current worst charge and discharge pairs
    # have breakeven, all remaining pair will also exceed breakeven.
    while DP < Breakeven(CP):
        
        # The variable 'position' tells the code on which comparison to make
        # First: compares the i-th action against i-th competing action
        if position == "First":
            
            # Define compared dispatches
            fres = first["Restrict"].loc[i]
            sres = second["Restrict"].loc[i]

            # Drop the action with least dispatch
            second = second.drop(i)
            
            # Update the action with greater dispatch
            first["Restrict"].loc[i] = fres + sres
            
            try:
                # Set next worst action as next comparison
                if pos == "Dis":
                    DP = first["Price"].loc[i + 1]
                else:
                    DP = second["Price"].loc[i + 1]
            except:
                # Exception if last entry
                break
            
            # Update position
            position = "Second"
            
            # Iteration only updates when position = 'First'
            i += 1
            
        # Second: Compares the i-th action and (i-1)-th competing action
        elif position == "Second":
            
            # Define compared dispatches
            fres = first["Restrict"].loc[i - 1]
            sres = second["Restrict"].loc[i]
            
            # Drop the action with least dispatch
            first = first.drop(i - 1)
            
            # Update the action with greater dispatch
            second["Restrict"].loc[i] = fres + sres
            
            try:
                # Set next worst action as next comparison
                if pos == "Dis":
                    CP = second["Price"].loc[i + 1]
                else:
                    CP = first["Price"].loc[i + 1]
            except:
                # Exception if last entry
                break
            
            # Update position
            position = "First"
            
        # Stop if any ROCA runs out of actions
        if min(len(first), len(second)) == 0:
            break
    
    # Retain original charging dataframe
    if pos == "Dis":
        charge = second
    else:
        charge = first

    if pos == "Dis":
        second = charge
        #first = first[first["Restrict"] != 0]
        #second = second[second["Restrict"] >= 0]
    else:
        first = charge 
        #first = first[first["Restrict"] >= 0]
        #second = second[second["Restrict"] != 0]
        
    # Concat all processed dataframes
    sdf = pd.concat([first, second])
    sdf["Status"] = np.where(sdf["Actual"] < 0, "Charge", "Discharge")
    
    return sdf

'''
CompareROCA takes:
- sdf as the processed df after Moving Average and Region Maximisation model
loops through each rocas and removes the pairings that are considered below the break-even point
'''

def CompareROCA(sdf):
    
    # Find all unique ROCAs
    ROCAs = sdf["Period"].unique().tolist()
    dflist = []
    
    # Loop through all charging ROCAs
    for i in [x for x in ROCAs if x % 2 == 1]:
        subdf = sdf[(sdf["Period"] == i) | (sdf["Period"] == i + 1)]
        subdf = RemoveLoss(subdf) # Main algorithm function
        dflist.append(subdf)
        
    # Concating all processed sub-dataframes
    findf = pd.concat(dflist, ignore_index = True).sort_values("Time")
    
    return FillCapacity(findf)

In [17]:
'''
LossRemoval takes:
- sdf as the processed df after Moving Average and Region Maximisation model
and returns the df after removing the pairings that are considered not profitable
'''

def LossRemoval(sdf):
    
    # Preprocessing
    rldf = LRProcess(sdf.copy())
    
    # Remove Losses
    rldf = CompareROCA(rldf)
    
    return rldf.drop("Period", axis = 1)

## <font color = "brown"> Stationary Maximisation Model</font>

The stationary maximisation (SM) model looks at all the time points where previous models ignores (which we will call stationary points) and ask the question, "Could there be revenue in these time points?". To understand how this model works,  we need to know how the revenue calculation works.


When charging takes place in a time point $t$, at $C$ Mwh, the actual increase in capacity is only $D = C \times 0.9$ accounting for efficiency. When dispatching this in the next time point, the maximum possible dispatched amount is $D \times 0.9$ accounting to efficiency. At the end, the revenue from charge $C$ is only obtained from $C \times 0.9^2$.

Making use of this efficiency fact, let $P_c$ be the theoretical charging price, $P_d$ be the theoritical discharging price and $R$ be the revenue on any time point. If we want to know whether revenue can be obtained from stationary points, we have to calculate the theoritical revenue, given that we know both charge and discharge prices.

$$
\begin{aligned}
R &= D P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
&=  0.9^2 C P_d \times 0.991 - C P_c \times \frac{1}{0.991} \\
&= (0.80271 P_d -  \frac{P_c}{0.991})C
\end{aligned}
$$

Notice that the calculation for revenue is now a scalar multiple of the initial charging dispatch. This means that we can just set $C = 1$ and see if the revenue is positive or not to know whether any other value of $C > 0$ is also profitable. Hence the SM model calculates the theoretical revenue,

$$
\begin{aligned}
TR &= 0.80271 P_d -  \frac{P_c}{0.991}
\end{aligned}
$$

Where $P_c = P_t$ and $P_d = P_{t+1}$, in which $t$ indicates the current time iteration. The model loops for every stationary point $t$, and assign charging and discharging status respectively at time $t$ and $t+1$ if the $TR$ at time $t$ is $>0$. Because the revenue is a scalar multiple of $C$, we can maximise the revenue by maximising $C$, hence the amount of dispatch will be the maximum possible given the current opening capacity at time $t$.

In [18]:
'''
Prepare takes:
- df, the original market data
- n, the dataframe after models preprocessing
and returns the df containing all data and with no missing values
'''

def Prepare(df, N):
    
    # Simple preprocessing
    N = N[N["Actual"] != 0]
    
    ND = pd.merge(df, N[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]], 
                  on = ["Time", "Price"], how = "left")
    
    # Imputation
    ND["Status"] = ND["Status"].replace(np.nan, "Do Nothing")
    ND["Actual"] = ND["Actual"].replace(np.nan, 0)
    ND["Restrict"] = ND["Restrict"].replace(np.nan, 0)
    
    # Fix Capacities
    return FillCapacity(ND)

In [19]:
'''
Profit takes:
- D, the discharge price
- C, the charge price
- charge, the charging dispatch
and returns the theoritical profit
'''

def Profit(D, C, charge):
    
    # Function to find theoritical profit given the 
    # charge dispatch, charge price, and discharge price
    discharge = charge * 0.9 * 0.9
    profit = D * discharge * 0.991 - C * charge / 0.991
    
    return profit

'''
SMProcess takes:
- df, the processed fataframe
and returns the df containing additional feature for this model
'''

def SMProcess(df):
    
    # Function to create features based on theoritical revenue calculations
    D = df.loc[df["Status"] == "Do Nothing"].sort_values("Time")
    D["Datediff"] = (D["Time"] - D["Time"].shift()).dt.total_seconds()/60
    D["NextPrice"] = D["Price"].shift(-1)
    D["Theoritical"] = Profit(D["NextPrice"], D["Price"], 150)
    
    return D

In [20]:
'''
LambdaLimit takes:
- capacity
- status
and returns the maximum possible dispatch if status is charging
'''
def LambdaLimit(capacity, status):
    
    # Assign Restricted Dispatch
    if status == "Charge":
        return abs(Limit(capacity, status))
    else:
        return 0
    
'''
InterchangeCheck takes:
- n, an entry of the list
- ls, the entire list
and returns true if (n-1) in list and (n-2) not in list
'''  

def InterchangeCheck(n, ls):
    # Check for consecutive charges
    if (n - 1) in ls:
        # Allow if number of 
        # consecutive charge exceeds 2
        if (n - 2) in ls:
            return True
        else:
            return False
        
    # If no consecutive charge, no actions
    else:
        return True
    
'''
Reassign takes:
- D, a dataframe
and returns the dataframe with reassigned statuses.
'''

def Reassign(D):
    
    # Assign possible charging conditions
    D["Status"] = np.where((D["Theoritical"] > 0) & (D["Datediff"].shift(-1) == 30) & (D["Opening Capacity"] != 580), 
                            "Charge", "Do Nothing")
    D["Previous Status"] = D["Status"].shift()
    
    # Find charging indexes
    charges = D[D["Status"] == "Charge"].index
    
    # Filter indexes for consecutive entries
    while(len([x for x in charges if x + 1 in charges]) != 0):
        charges = [x for x in charges if InterchangeCheck(x, charges)]
        
    # Initialise charges
    D["Index"] = D.index
    D["Status"] = np.where(D["Index"].isin(charges), "Charge", "Do Nothing")
    
    # Apply restrited dispatches
    D["Restrict"] = D.apply(lambda x: LambdaLimit(x["Opening Capacity"], x["Status"]), axis = 1)
    D["Previous"] = D["Status"].shift()
               
    # Initialise discharges and its restrictions
    D["Status"] = np.where(D["Previous"] == "Charge", "Discharge", D["Status"])
    D["Restrict"] = np.where(D["Status"] == "Discharge", -D["Restrict"].shift(), D["Restrict"])
    
    # Filter
    D = D[D["Status"] != "Do Nothing"]
    
    return D, D.index

In [21]:
'''
SMPost takes:
- Station, the dataframe containing stationary points
- Model, the dataframe with non-stationary points
- Index, the index of Station in the original dataframe
and returns the concated dataframe
'''

def SMPost(Station, Model, index):
    
    # Concating the results from this model with previous model
    Fin = pd.concat([Model.drop(index), Station[Model.columns]]).sort_values("Time")
    Fin["Actual"] = - np.where(Fin["Status"] == "Discharge", Fin["Restrict"] * 0.9, Fin["Restrict"] / 0.9)
    
    return Fin[['Time', 'Price', 'Status', 'Actual', 'Restrict', 
                'Opening Capacity', 'Closing Capacity']]

In [22]:
'''
Statinary takes:
- N, dataframe from previous model
- df, the original dataframe
and returns the df after stationary processing
'''

def Stationary(N, df):
    
    # Preprocessing
    Model = Prepare(df, N)
    
    # Feature Engineering
    Station = SMProcess(Model)
    
    # Algorithm to find theoritically profitable transactions
    Station, index = Reassign(Station)
    
    # Post Process
    Findf = SMPost(Station, Model, index)
 
    return Findf

## <font color = "brown"> Action Shift Model </font>

The action shift (AS) model is an extension of the RM model. The RM model allocates most dipatches to the best prices in a ROCA. The AS model shifts charging or discharging actions to the best price in a region of consecutive non-actions and actions (ROCNA). 

A ROCNA is classified into a charging ROCNA and a discharging ROCNA. The formal definition of a ROCNA is a region between 2 opposing actions where at least 1 non-opposing action is present. i.e Let's say that we want to look for a charging ROCNA at any point in the data. If a discharge action is present at time $t$ and the next discharging action is located at time $t + n$, then the period $[t + 1, t + n -1]$ is classified as a charging ROCNA period, if and only if there exist at least $1$ charging point in the region. This is also applicable for a discharging ROCNA, where we are now looking for periods between 2 charging actions non-inclusive and a discharge between. Note that a time point is not exclusive to one ROCNA, as it can be part of 2 ROCNAs maximum. The table below shows the movement of some ROCNAs.

<table>
    <tr><th> Time </th><th> Action </th><th>  ROCNACharge </th><th>  ROCNADischarge </th></tr>
    <tr><td> 2120-01-01 00:00:00 </td><td> Charge </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 00:30:00 </td><td> Do Nothing </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 01:00:00 </td><td> Charge </td><td> 1 </td><td>  </td></tr>
    <tr><td> 2120-01-01 01:30:00 </td><td> Do Nothing </td><td> 1 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 02:00:00 </td><td> Discharge </td><td>  </td><td> 1 </td></tr>
    <tr><td>2120-01-01 02:30:00 </td><td> Discharge </td><td>  </td><td> 1 </td></tr>
    <tr><td>2120-01-01 03:00:00 </td><td> Do Nothing </td><td> 2 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 03:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 1 </td></tr>
    <tr><td>2120-01-01 04:00:00 </td><td> Charge </td><td> 2 </td><td>  </td></tr>
    <tr><td>2120-01-01 04:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 05:00:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 05:30:00 </td><td> Do Nothing </td><td> 2 </td><td> 2 </td></tr>
    <tr><td>2120-01-01 06:00:00 </td><td> Discharge </td><td>  </td><td> 2 </td></tr>
    <tr><td>2120-01-01 06:30:00 </td><td> Do Nothing </td><td>  </td><td> 2 </td></tr>
</table>

The SA model looks at each ROCNAs chronologically by their starting time and shift around the actions to their maximised version. In order to preserve capacity level, a restriction must be put in place after every ROCNA shifts. If a shift takes place in the previous opposing ROCNA, then the time points considered for the current iteration's maximisation can only take place in the time points above the maximum (time) action in the previous ROCNA shifts.

In mathematical terms, if a single ROCNA $R_k$ has actions on time $\{a, b, c, d, e\}$ and goes through the SA model,

$$
\begin{aligned}
\{y_a, y_b, y_c, y_d, y_e\} &\xrightarrow[\text{}]{\text{Shift}} \{y_{a'}, y_{b'}, y_{c'}, y_{d'}, y_{e'}\}
\end{aligned}
$$

To analyse the next ROCNA $R_{k+1}$, we want to look at the maximum time of the shifted actions,

$$
\begin{aligned}
T &= \text{Max}_{\text{Time}}(y_{a'}, y_{b'}, y_{c'}, y_{d'}, y_{e'})
\end{aligned}
$$

Then, analysing $R_{k+1}$ can not be done naïvely on the entire ROCNA due to capacity limitations. To maintain capacity restrictions, the next shifting region takes place only on the time points $t$ that satisfies,

$$
\begin{aligned}
(t \in R_{k+1}) \cap (t > T)
\end{aligned}
$$

This process is iterated for all charging and discharging ROCNAs.

In [23]:
'''
ROCNA takes:
- df, the dataframe in question
- status, the current region assessed
- other, the current region not assessed
and returns the df with assigned ROCNA
'''

def ROCNA(df, status, other):

    # Finding ROCNA periods
    W = df[df["Status"] == status]["Period"].unique()
    W = np.sort(np.unique(np.array(list(W) + list(W+1) + list(W-1))))
    
    # Finding ROCNA limitations
    Banned = df[df["Status"] == other]["Period"].unique()
    W = W[~np.isin(W, Banned)]

    # Merge ROCNA column to initial dataframe
    Com = pd.DataFrame()
    Com["Period"] = W
    Com[f"ROCNA{status}"] = ((Com["Period"] - Com["Period"].shift()) != 1).cumsum()
    fin = pd.merge(df, Com, on = "Period", how = "left")

    return fin

'''
ASProcess takes:
- df, dataframe from previous models
and returns the df after column filtering and feature creation
'''

def ASProcess(df):
    
    # Assign ROCA
    df = FillCapacity(df)
    df = df[["Time", "Price", "Status", "Actual", "Restrict", "Opening Capacity", "Closing Capacity"]]
    df["Period"] = df["Status"].ne(df["Status"].shift()).cumsum()
    
    # Assign ROCNA from ROCAs
    Actions = ["Charge", "Discharge"]
    for i in range(len(Actions)):
        df = ROCNA(df, Actions[i], Actions[i - 1])
    
    # Create column for maximum time restrictions
    df["Shift"] = False
    
    return df

In [24]:
'''
Shift takes:
- df, a dataframe
and returns maximised ROCNA version
'''

def Shift(df):
    
    # Main algorithm function
    
    # Iteration variable for each 
    # charging and discharging ROCNA
    i, j = 1, 1
    
    # First ROCA is always charging
    current = "Charge"
    
    # Loop until reaching last charge and discharge ROCNAs
    while (i < df["ROCNACharge"].max() and j < df["ROCNADischarge"].max()):
        
        # Current ROCNA action
        stat = current
        
        # Action for charging ROCNA
        if current == "Charge":
            
            # ROCNACharge: Filtering iteration's charging ROCNA
            # Shift: Filter time points after maximum time shift
            subdf = df[(df["ROCNACharge"] == i) & (~df["Shift"])]
            
            # Sort by best prices to worst (lowest to highest)
            subdf = subdf.sort_values("Price", ascending = True)
            
            # Increase charge iteration
            i += 1
            
            # Change next action
            current = "Discharge"
            
        # Action for discharging ROCNA
        elif current == "Discharge":
            # ROCNACharge: Filtering iteration's discharging ROCNA
            # Shift: Filter time points after maximum time shift
            subdf = df[(df["ROCNADischarge"] == j) & (~df["Shift"])]
            
            # Sort by best prices to worst (highest to lowest)
            subdf = subdf.sort_values("Price", ascending = False)
            
            # Increase charge iteration
            j += 1
            
            # Change next action
            current = "Charge"
            
        # Skip next code if length of ROCNA is 1
        if len(subdf) == 1:
            continue
            
        # Shift best dispatches to best prices
        Dispatches = sorted([abs(x) for x in subdf["Restrict"].tolist()], reverse = True)
        subdf["Restrict"] = Dispatches
        subdf["Status"] = np.where(subdf["Restrict"] > 0, stat, "Do Nothing")
        
        # Record maximum time after shift
        subdf["Shift"] = np.where((subdf["Time"] <= subdf[subdf["Restrict"] > 0]["Time"].max())
                                  , True, False)
        
        # Replace ROCNA with shifted ROCNA
        df.iloc[subdf.index] = subdf
       
    return df

In [25]:
'''
ASPost takes:
- df, a dataframe
and returns the df containing the correct dispatch calculation
'''

def ASPost(df):
    
    # Fix dispatch and efficiency calculation and sign
    df["Restrict"] = np.where(df["Status"] == "Discharge", -abs(df["Restrict"]), abs(df["Restrict"]))
    df["Actual"] = - np.where(df["Status"] == "Discharge", df["Restrict"] * 0.9, df["Restrict"] / 0.9)
    
    return df.sort_values("Time")

In [26]:
'''
ShiftAction takes:
- sdf, dataframe from previous models
- df, the original data
and returns the df after model processing
'''

def ShiftAction(sdf, df):
    
    # Assign ROCNAs
    sdf = pd.merge(df, sdf[["Time", "Price", "Status", "Actual", "Restrict", 
                            "Opening Capacity", "Closing Capacity"]], 
                   on = ["Time", "Price"], how = "left")
    SA = ASProcess(sdf.copy())
    
    # Shift actions in ROCNAs
    SA = Shift(SA)
    
    # Post processing
    SA = ASPost(SA)
    
    # Fill capacities after shifting
    SA = FillCapacity(SA)
    
    return SA

## <font color = "brown"> Miscellaneous </font>

In [27]:
'''
CalculateRevenue takes:
- df, a dataframe containing dispatches
and returns the revenue of the dataframe
'''

# This function calculates the revenue by multiplying its dispatch, mlf and price columsn
def CalculateRevenue(df):
    
    # Remove stationary points
    rev = df[(df["Status"] == "Charge") | (df["Status"] == "Disharge")].sort_values("Time")
    
    # Set marginal loss factors for each action
    df["mlf"] = np.where(df["Status"] == "Charge", 1/0.991, 0.991)
    
    # Ignoring signs
    df["Actual"] = np.where(df["Status"] == "Charge", -abs(df["Actual"]), abs(df["Actual"]))
    
    # Find revenue for each actions
    df["Revenue"] = df["Price"] * df["Actual"] * df["mlf"]
    
    # Sum up revenues
    revenue = df["Revenue"].sum()
    
    return revenue

In [28]:
'''
ValidityCheck takes:
- df, a dataframe containing the complete packages
and returns a validity checker
'''

# This function takes processed df and checks if there is on error in capacity
def ValidityCheck(df):
    
    # Check if power used exceeds 300
    Condition0 = abs(df["Actual"].max()) > 150
    
    # Check if capacity exceeds 580
    Condition1 = df["Closing Capacity"].max() > 580
    
    # Check if capacity falls below 0
    Condition2 = df["Closing Capacity"].min() < 0
    
    # Check for duplicate time
    Condition3 = (len(df) != len(df.drop_duplicates("Time")))
    
    # Check for non-continuous time
    Condition4 = ((((df["Time"] - df["Time"].shift()).dt.total_seconds()/60) < 0).sum()) != 0
    
    # Check capacities validness
    Condition5 = (df["Opening Capacity"] != df["Closing Capacity"].shift()).sum() > 1
    
    # Decreasing charge
    Condition6 = len(df[(df["Status"] == "Charge") & ((df["Closing Capacity"] - df["Opening Capacity"]) < 0)]) != 0
    
    # Increasing discharge
    Condition7 = len(df[(df["Status"] == "Discharge") & ((df["Closing Capacity"] - df["Opening Capacity"]) > 0)]) != 0
    
    # Printed error messages
    ErrorMessages = ["Power Exceeds 300!",
                     "Capacity Exceeds 580!",
                     "Capacity Below 0!",
                     "Duplicate Records Found!",
                     "Non-Consecutive Time Data!",
                     "Capacities Chain Broken!",
                     "Charging Decreases Capacity!",
                     "Discharging Increases Capacity!"]
    
    # Boolean error list
    Errors = [Condition0, Condition1, Condition2, Condition3, 
              Condition4, Condition5, Condition6, Condition7]
    
    count = 0
    # Loop through all possible errors
    for e, m in zip(Errors, ErrorMessages):
        if e:
            # Print if error is true
            print(m)
            count += 1
            
    # Number of errors
    print(f"Number of Errors: {count}")

## <font color = "brown"> Main </font>

In [36]:
'''
main takes:
- df as the energy market raw data
processes these into the main models
and returns the df containing the actions when to charge or discharge
'''

def main(df, n = 17):
    
    Model = MovingAverage(n, df)
    Model = Maximisation(Model, df)
    Model = LossRemoval(Model)
    Model = Stationary(Model, df)
    Model = ShiftAction(Model, df)

    return Model

In [37]:
'''
sandwich takes:
- df as the entire data set after being passed the first 5 mini models
adds more layers to the models, and returns the final model
'''
def sandwich(model, df):
    
    Model = Maximisation(model, df, Chronos = False)
    Model = Stationary(Model, df)
    Model = ShiftAction(Model, df)

    return Model